In [ ]:
#This ipynb is using Yolov5_VGG_SVM , taking first frame from the start
#taking first frame and all frame to calculate the max show ups cattle id
#No black box

In [ ]:
# https://youtu.be/9GzfUzJeyi0

import os
import numpy as np
import cv2
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

"""
@author: Sreenivas Bhattiprolu
"""

import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import torch
import torch.backends.cudnn as cudnn

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
#from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
from tensorflow.keras.applications.vgg16 import VGG16


total_classes=0  #to change when mee lay want to add more cows
#images_per_validation=5


#print(os.listdir("../Gold"))

SIZE = 224

import csv

#data = pd.read_csv('D:\\LamenessData\\September_6\\evening.csv',index_col ='Local_ID')

#COW_MAPPER = [list(row) for row in data.values]

#train_images = []
#train_labels = []

base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(SIZE, SIZE, 3))

train_images = []
train_labels = []
for directory_path in glob.glob("D:\LamenessData\yolov8_features\\yolov8_segment_dataset\\Training/*"):
    label = directory_path.split("\\")[-1]
    #print(label)
    #total_classes+=1
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        #img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        
        
        #img = cv2.resize(img, (SIZE, SIZE))
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        #train_images.append(img)
        #train_labels.append(label)
        
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (SIZE, SIZE))
        image = preprocess_input(image)
        feature = base_model.predict(np.array([image]))[0]
        train_images.append(feature)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

#print(lables.len)
#nb_classifier = GaussianNB()
#nb_classifier.fit(train_images, labels)





C:\Users\thithilab\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1/1 [==============================] - 0s 54ms/step


In [ ]:
test_images = []
test_labels = []
for directory_path in glob.glob("D:\LamenessData\yolov8_features\\yolov8_segment_dataset\\Validation/*"):
    label = directory_path.split("\\")[-1]
    #print(label)
    #total_classes+=1
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        #img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        
        
        #img = cv2.resize(img, (SIZE, SIZE))
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        #train_images.append(img)
        #train_labels.append(label)
        
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (SIZE, SIZE))
        image = preprocess_input(image)
        feature = base_model.predict(np.array([image]))[0]
        test_images.append(feature)
        test_labels.append(label)
        
test_images = np.array(test_images)
test_labels = np.array(test_labels)


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:

x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded
x_train, y_train = train_images,train_labels_encoded#, test_images, test_labels_encoded

###################################################################
# Normalize pixel values to between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

In [ ]:
import pickle
lcnn_x_train_name = 'June_Weights_LCNN//lcnn_x_train.pkl'
pickle.dump(x_train, open(lcnn_x_train_name, 'wb'), protocol=4)

lcnn_y_train_name ='June_Weights_LCNN//lcnn_y_train.pkl'
pickle.dump(y_train,open(lcnn_y_train_name,'wb'), protocol=4)


In [ ]:
print('done')

In [ ]:
#np_features = np.array(features)

In [ ]:
from sklearn.naive_bayes import GaussianNB
#extracted_features = np_features.reshape(np_features.shape[0], -1)
nb_classifier = GaussianNB()
nb_classifier.fit(x_train, y_train)

In [ ]:
import pickle
#saving VGG and SVM trained models

#base_model.save('May_Weights/LCNN_GNB/all_cows_lcnn_v1.h5')


#le_filename = 'May_Weights/LCNN_GNB/all_cows_LE_v1.le'
#pickle.dump(le, open(le_filename, 'wb'))

nb_classifier_name ='May_Weights/LCNN_GNB/all_cows_GNB_v1.pkl'
pickle.dump(nb_classifier,open(nb_classifier_name,'wb'))


#base_model_name ='May_Weights/LCNN_GNB/all_cows_LCNN_v1.h5'
#pickle.dump(base_model,open(base_model_name,'wb'))

print("done")


#########################



In [ ]:
#SVM

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV

#X_train, X_test, y_train, y_test = train_test_split( features, 
#                                                     labels, 
#                                                     test_size=0.30)
                                         
    
clf = LinearSVC(random_state=21, tol=0.0005,max_iter=1500) #rs=10 maxiter=500  tol=0.01  
#clf = CalibratedClassifierCV(clf)  #added
clf.fit(x_train, y_train)
print("done")

#print(x_test.shape)
#x_test_svm=x_test.reshape(32786,256)
#x_test_svm = (x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3]))
#target_x = (target_x.reshape(target_x.shape[0], target_x.shape[1] * target_x.shape[2] * target_x.shape[3]))



In [ ]:

svm_classifier_name ='May_Weights/LCNN_SVM/all_cows_SVM_v1.pkl'
pickle.dump(clf,open(svm_classifier_name,'wb'))

print("done")

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 300, random_state = 72) #75-68%  #100 86%  #72-93%
print("defined rf")
# Train the model on training data
RF_model.fit(x_train, y_train) #For sklearn no one hot encoding
print("RF_model.fit is running")


In [ ]:

rf_classifier_name ='May_Weights/LCNN_RF/all_cows_RF_v1.pkl'
pickle.dump(RF_model,open(rf_classifier_name,'wb'))

print("done")

In [ ]:
print(total_classes)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)


In [ ]:
knn.fit(x_train, y_train)

In [ ]:
knn_filename ='May_Weights/LCNN_KNN/all_cows_KNN_p4_v1.pkl'
pickle.dump(knn,open(knn_filename,'wb'), protocol=4)


In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_for_RF, y_train)


In [ ]:
x_test = test_images/ 255.0

In [ ]:
import lightgbm as lgb

lgb_train = lgb.Dataset(x_train, y_train)
val_data = lgb.Dataset(x_test, label=y_test, reference=lgb_train)
# Set LightGBM parameters
params = {
    'objective': 'multiclass',
    'num_class': 143,  # Replace 'num_classes' with the actual number of classes in your dataset
    'metric': 'multi_logloss',
    'learning_rate':0.05,
    #'device_type': 'cuda'
    'early_stopping_rounds': 10,
    'force_col_wise':True
}

# Train the LightGBM model


In [ ]:
model = lgb.train(params, lgb_train, valid_sets=val_data)


In [ ]:
import pickle
model_name ='May_Weights/LCNN_LIGHTGBM/all_cows_LIGHTGBM_v1.pkl'
pickle.dump(model,open(model_name,'wb'))

print("done")

In [ ]:
import pickle
#saving VGG and SVM trained models

VGG_model.save('May_Weights/all_cows_vgg_v1.h5')


le_filename = 'May_Weights/all_cows_le_v1.le'
pickle.dump(le, open(le_filename, 'wb'))

gnb_filename ='May_Weights/all_cows_svm_v1.pkl'
pickle.dump(gnb,open(gnb_filename,'wb'))

print("done")


#########################



In [ ]:
#load modelsavedModels
#vgg_filename = 'vgg16_model.sav'
#svm_filename ='svm.sav'
#VGG_model = pickle.load(open(vgg_filename, 'rb'))
#clf = pickle.load(open(svm_filename, 'rb'))

In [ ]:
#One class SVM
#model =  OneClassSVM(nu=0.1, degree=3, kernel='rbf',gamma='scale')
#data_new_clean = known_images.loc[known_images.total_count <=known_images.total_count.quantile(1-outlier_fraction)]
#model.fit(X_for_oneclass)
print("done")

In [ ]:
#run second part here

In [ ]:
#cv2.destroyAllWindows()

In [ ]:
#ipkernel_launcher.py --source "D:\Python\env\Lameness\Frames\Videos\20220201_145508_7108.mp4"  --yolo-weights weights_slm/best_6_23_gpu.pt --view-img --save-crop --device 0

In [ ]:
#python -h

In [ ]:
#import cv2
#print(cv2. __version__) # previous version 4.6.0

In [ ]:
#p.set_printoptions(threshold=sys.maxsize)  #run me before printing , to find the meaning of variable, you can just find and print them

In [ ]:
#print(X_test_feature) #print me

In [ ]:
#print(X_test_features)  #print me

In [ ]:
#print(X_For_RF) ## print me